# Simularium Conversion Tutorial : Custom Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio import CustomConverter, CustomData, AgentData, UnitData

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium custom data Converter consumes spatiotemporal data from any source. It requires a `CustomData` object as parameter.

`CustomData` contains the following:
* **box_size** : *np.ndarray (shape = \[3\])*
    * A numpy ndarray containing the XYZ dimensions of the simulation bounding volume
* **agent_data** : `AgentData`
    * An object containing data for each agent at each timestep
* **time_units**: `UnitData` *(optional)*
    * multiplier and unit name for time values
    * Default: 1.0 second
* **spatial_units**: `UnitData` *(optional)*
    * multiplier and unit name for spatial values
    * Default: 1.0 meter
* **plots** : *List\[Dict\[str, Any\]\] (optional)*
    * An object containing plot data already in Simularium format
    
`AgentData` contains the following:
* **times** : *np.ndarray (shape = [timesteps])*
    * A numpy ndarray containing the elapsed simulated time at each timestep (in the time units specified by **CustomData.time_unit_factor_seconds**)
* **n_agents** : *np.ndarray (shape = [timesteps])*
    * A numpy ndarray containing the number of agents that exist at each timestep
* **viz_types** : *np.ndarray (shape = [timesteps, agents])*
    * A numpy ndarray containing the viz type for each agent at each timestep. Current options:
        * 1000 : default,
        * 1001 : fiber (which will require subpoints)
* **unique_ids** : *np.ndarray (shape = [timesteps, agents])*
    * A numpy ndarray containing the unique ID for each agent at each timestep
* **types** : *List[List[str]] (list of shape [timesteps, agents])*
    * A list containing timesteps, for each a list of the string name for the type of each agent
* **positions** : *np.ndarray (shape = [timesteps, agents, 3])*
    * A numpy ndarray containing the XYZ position for each agent at each timestep
* **radii** : *np.ndarray (shape = [timesteps, agents])*
    * A numpy ndarray containing the radius for each agent at each timestep
* **n_subpoints** : *np.ndarray (shape = [timesteps, agents]) (optional)*
    * A numpy ndarray containing the number of subpoints belonging to each agent at each timestep. Required if subpoints are provided
    * Default: None
* **subpoints** : *np.ndarray (shape = [timesteps, agents, subpoints, 3]) (optional)*
    * A numpy ndarray containing a list of subpoint position data for each agent at each timestep. These values are currently only used for fiber agents
    * Default: None
* **draw_fiber_points** : *bool (optional)*
    * Draw spheres at every other fiber point for fibers?
    * Default: False

`UnitData` contains the following:
* **name**: *str*
    * unit name for values (we support this list https://github.com/hgrecco/pint/blob/master/pint/default_en.txt)
* **magnitude**: *float (optional)*
    * multiplier for values (in case they are not given in whole units)
    * Default: 1.0

### Generate example data

To demonstrate using the custom converter, we'll first generate some random example data.

#### If there are NO fibers in the data:

In [2]:
from string import ascii_uppercase
from random import choice

# parameters
total_steps = 5
timestep = 0.5
box_size = 100
n_agents = 5
min_radius = 5
max_radius = 10

type_names = []
for t in range(total_steps):
    type_names.append([choice(ascii_uppercase) for i in range(n_agents)])
    
example_default_data = CustomData(
    box_size=np.array([box_size, box_size, box_size]),
    agent_data=AgentData(
        times=timestep * np.array(list(range(total_steps))),
        n_agents=np.array(total_steps * [n_agents]),
        viz_types=np.array(total_steps * [n_agents * [1000.0]]),  # default viz type = 1000
        unique_ids=np.array(total_steps * [list(range(n_agents))]),
        types=type_names,
        positions=np.random.uniform(size=(total_steps, n_agents, 3)) * box_size - box_size * 0.5,
        radii=(max_radius - min_radius) * np.random.uniform(size=(total_steps, n_agents)) + min_radius
    ),
    time_units=UnitData("ns"),  # nanoseconds
    spatial_units=UnitData("nm"),  # nanometers
)

#### Otherwise:

In [3]:
from string import ascii_uppercase
from random import choice

# parameters
total_steps = 5
timestep = 0.5
box_size = 100
n_agents = 5
min_radius = 5
max_radius = 10
points_per_fiber = 4

type_names = []
for t in range(total_steps):
    type_names.append([choice(ascii_uppercase) for i in range(n_agents)])

example_fiber_data = CustomData(
    box_size=np.array([box_size, box_size, box_size]),
    agent_data=AgentData(
        times=timestep * np.array(list(range(total_steps))),
        n_agents=np.array(total_steps * [n_agents]),
        viz_types=np.array(total_steps * [n_agents * [1001.0]]), # fiber viz type = 1001
        unique_ids=np.array(total_steps * [list(range(n_agents))]),
        types=type_names,
        positions=np.zeros(shape=(total_steps, n_agents, 3)),
        radii=np.ones(shape=(total_steps, n_agents)),
        n_subpoints=points_per_fiber * np.ones(shape=(total_steps, n_agents)),
        subpoints=box_size * np.random.uniform(
            size=(total_steps, n_agents, points_per_fiber, 3)) - box_size * 0.5
    ),
    time_units=UnitData("ns"),  # nanoseconds
    spatial_units=UnitData("nm"),  # nanometers
)

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_default_data` or `example_fiber_data` object, you can use the converter to generate the file at the given path:

In [4]:
CustomConverter(example_fiber_data).write_JSON("example_fibers")
CustomConverter(example_default_data).write_JSON("example_default")

Writing JSON -------------
Reading Custom Data -------------
saved to example_fibers.simularium
Writing JSON -------------
Reading Custom Data -------------
saved to example_default.simularium


## Visualize in the Simularium viewer

In a supported web-browser (Firefox or Chrome), navigate to https://simularium.allencell.org/ and import your file into the view.